In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
tf.ones([1, 3], tf.int32)

<tf.Tensor: id=2, shape=(1, 3), dtype=int32, numpy=array([[1, 1, 1]], dtype=int32)>

## Old stuff

In [4]:
class GetEmbeddings(tf.keras.layers.Layer):
    def __init__(self, num_buckets=None, embedding_size=None,*args, **kwargs):
        super(GetEmbeddings, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.embedding = tf.constant([[1.0,1.0,1.0,1.0],[2.0,2.0,2.0,2.0],[3.0,3.0,3.0,3.0],[4.0,4.0,4.0,4.0]]) # Embedding matrix
    
    def call(self, input):
        user = input[0]
        movie = input[1]
        # Expand user to get it to a matrix of 1 and zeros instead
        user = tf.expand_dims(user, 2)
        user = tf.tile(user, [1,1, 4])
        # check equality of array in t1 for each array in t2 element by element. This is possible because the equal function supports broadcasting. 
        equal =  tf.math.equal(tf.constant([1.0,1.0,1.0,1.0]), user)
        # checking if all elements from  t1 match for all elements of some array in t2
        equal_all = tf.reduce_all(equal, axis=2)
        user_embeddings = self.embedding * user 
        # Get embeddings for the users based upon the movies he/she watched
        # Expand the movie first
        movies = tf.expand_dims(movie,2)
        movies = tf.tile(movies, [1,1, 4])
        # Get the embeddings
        movie_embeddings = self.embedding * movies
        # Only want the movies with values
        equal_movie =  tf.math.not_equal(tf.constant([0.0,0.0,0.0,0.0]), movie_embeddings)
        equal_all_movie = tf.reduce_all(equal, axis=1)
        # Get the indicies where we have a movie
        indices = tf.where(equal_movie)
        # In order to to the pointwise multiplication I need to have the same shape so expand
        out = tf.reshape(tf.gather_nd(movie_embeddings,indices),shape=(2,1,4))
        out = tf.tile(out,multiples=[1,4,1])
        # Point wise multiplication, each movie the user watched multiplied with this movie
        output = tf.math.multiply(out,user_embeddings)
        lenght =  tf.reduce_sum(tf.cast(equal_all, tf.float32),axis=1)
        mean = tf.reduce_sum(tf.cast(output,dtype=tf.float32),axis=1) / tf.reshape(lenght, (-1, 1))
        return mean

## Custom layer

In [350]:
class GetEmbeddings(tf.keras.layers.Layer):
    def __init__(self, num_buckets=None, embedding_size=None,*args, **kwargs):
        super(GetEmbeddings, self).__init__(*args, **kwargs)
        self.movies = num_buckets
        self.embedding_size = embedding_size
        #self.shape = compute_output_shape
    
    def build(self,input_shape):
        self.embedding = tf.constant([[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]) # Embedding matrix
        self.kernel = self.add_variable("kernel",
                                    shape=[self.movies,
                                           self.embedding_size],dtype=tf.float32,initializer=tf.keras.initializers.RandomUniform(),trainable=True)
        
    def call(self, input):
        #input = tf.reshape(input)
        #movies_new = concated[:,:4]
        #users_new = concated[:,4:]
        user = input[:,:self.movies]
        movie = input[:,self.movies:]
        # Expand user to get it to a matrix of 1 and zeros instead
        user = tf.expand_dims(user, 2)
        user = tf.tile(user, [1,1, self.embedding_size])
        # check equality of array in t1 for each array in t2 element by element. This is possible because the equal function supports broadcasting. 
        #equal =  tf.math.equal(tf.constant([1,1,1,1]), user)       
        equal =  tf.math.equal(tf.ones([1, self.embedding_size]), user)
        # checking if all elements from  t1 match for all elements of some array in t2
        equal_all = tf.reduce_all(equal, axis=2)      
        #user_embeddings = self.embedding * user 
        user_embeddings = self.kernel * user
        # Get embeddings for the users based upon the movies he/she watched
        # Expand the movie first
        movies = tf.expand_dims(movie,2)
        movies = tf.tile(movies, [1,1, self.embedding_size])
        # Get the embeddings
        #movie_embeddings = self.embedding * movies      
        movie_embeddings = self.kernel * movies        
        # Only want the movies with values 
        equal_movie =  tf.math.not_equal(tf.zeros([1, self.embedding_size]), movie_embeddings)
        #print(equal_movie)
        equal_all_movie = tf.reduce_all(equal, axis=1)
        #print(equal_all_movie)
        # Get the indicies where we have a movie
        indices = tf.where(equal_movie)
        # In order to to the pointwise multiplication I need to have the same shape so expand
        #print(tf.gather_nd(movie_embeddings,indices))
        out = tf.reshape(tf.gather_nd(movie_embeddings,indices),shape=(tf.shape(user)[0],1,self.embedding_size))
        out = tf.tile(out,multiples=[1,self.movies,1])
        #print(tf.math.multiply(out[1],user_embeddings))
        # Point wise multiplication, each movie the user watched multiplied with this movie
        output = tf.math.multiply(user_embeddings,out)
        #print(output)
        #print(tf.shape(output))
        #output = tf.reshape(output,shape=(self.batch_size,self.movies,self.embedding_size))
        mean = tf.reduce_sum(tf.cast(output,dtype=tf.float32),axis=1) / self.movies
        return mean

In [364]:
layer = GetEmbeddings(4,10)
user = tf.constant([[1.0,1.0,1.0,1.0],[1.0,0.0,0.0,0.0],[1.0,0.0,0.0,0.0],[1.0,0.0,0.0,0.0]]) # Example user, two users
#PROBLEMS PROBLEMS 
movie = tf.constant([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,1.0,0.0]]) 
concated = tf.concat([tf.cast(user,dtype=tf.int32),tf.cast(movie,tf.int32)],axis=1)
concated = tf.cast(concated,tf.float32)
layer(concated)

<tf.Tensor: id=10496, shape=(4, 10), dtype=float32, numpy=
array([[-1.16437695e-05, -3.88579210e-05,  7.73757289e-04,
        -2.47777323e-04,  1.94154534e-04,  6.78373792e-04,
         2.05216475e-05,  7.09847081e-04,  6.54163159e-05,
         8.94898585e-06],
       [-2.02351166e-05,  2.57733132e-04, -3.31072835e-04,
        -5.22725386e-05,  1.72020082e-04,  2.98175273e-05,
         5.94616868e-05,  7.76321613e-05,  2.35067546e-05,
         2.04360604e-05],
       [ 2.27060518e-05, -3.96614138e-04,  2.16164044e-04,
        -3.01132415e-04,  9.11516981e-05,  2.85599177e-04,
        -2.70666569e-05,  1.97928952e-04, -5.09810598e-05,
         8.35147785e-06],
       [ 2.27060518e-05, -3.96614138e-04,  2.16164044e-04,
        -3.01132415e-04,  9.11516981e-05,  2.85599177e-04,
        -2.70666569e-05,  1.97928952e-04, -5.09810598e-05,
         8.35147785e-06]], dtype=float32)>

In [365]:
concated

<tf.Tensor: id=10432, shape=(4, 8), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)>

## Custom layer with dense layer connected

In [368]:
inputs = tf.keras.Input(shape=(8,)) # Don't forget the comma
x  = GetEmbeddings(4,10)(inputs)
x = tf.keras.layers.Dense(32,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [369]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_46 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
get_embeddings_173 (GetEmbed (None, 10)                40        
_________________________________________________________________
dense_54 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 33        
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


In [371]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.BinaryCrossentropy(),
              # List of metrics to monitor
              metrics=[tf.keras.metrics.BinaryAccuracy()])



In [384]:
x_train = concated
y_train = tf.constant([0,0,0,0])
x_val=x_train
y_val=x_val

In [386]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

Train on 4 samples, validate on 4 samples
Epoch 1/50
4/4 [==============================] - 0s 1ms/sample - loss: 0.6945 - binary_accuracy: 0.0000e+00 - val_loss: 0.6928 - val_binary_accuracy: 0.6562
Epoch 2/50
4/4 [==============================] - 0s 2ms/sample - loss: 0.6921 - binary_accuracy: 1.0000 - val_loss: 0.6925 - val_binary_accuracy: 0.6562
Epoch 3/50
4/4 [==============================] - 0s 779us/sample - loss: 0.6911 - binary_accuracy: 1.0000 - val_loss: 0.6921 - val_binary_accuracy: 0.6562
Epoch 4/50
4/4 [==============================] - 0s 1ms/sample - loss: 0.6899 - binary_accuracy: 1.0000 - val_loss: 0.6918 - val_binary_accuracy: 0.6562
Epoch 5/50
4/4 [==============================] - 0s 923us/sample - loss: 0.6888 - binary_accuracy: 1.0000 - val_loss: 0.6915 - val_binary_accuracy: 0.6562
Epoch 6/50
4/4 [==============================] - 0s 2ms/sample - loss: 0.6878 - binary_accuracy: 1.0000 - val_loss: 0.6912 - val_binary_accuracy: 0.6562
Epoch 7/50
4/4 [==========